# Warming levels

This Notebook explores the options provided in `xscen` to analyze climate simulations through the scope of global warming levels, instead of temporal horizons.

First, we just need to prepare some data. As in other notebooks, we'll use NorESM2-MM as our example dataset.

In [ ]:
# Basic imports
from pathlib import Path

import xarray as xr
import xesmf as xe
from matplotlib import pyplot as plt

import xscen as xs

output_folder = Path().absolute() / "_data"

project = {
    "title": "example-warminglevel",
    "description": "This is an example catalog for xscen's documentation.",
}

pcat = xs.ProjectCatalog(
    str(output_folder / "example-wl.json"),
    project=project,
    create=True,
    overwrite=True,
)

# Extract and regrid the data needed for the Tutorial

cat_sim = xs.search_data_catalogs(
    data_catalogs=[str(output_folder / "tutorial-catalog.json")],
    variables_and_freqs={"tas": "D"},
    other_search_criteria={"source": "NorESM2-MM", "activity": "ScenarioMIP"},
    periods=[[2000, 2050]],
    match_hist_and_fut=True,
)

region = {
    "name": "example-region",
    "method": "bbox",
    "tile_buffer": 1.5,
    "lon_bnds": [-68.5, -67.5],
    "lat_bnds": [48.5, 49.5],
}

ds_grid = xe.util.cf_grid_2d(-68.5, -67.5, 0.25, 48.5, 49.5, 0.25)
ds_grid.attrs["cat:domain"] = "region1"
for ds_id, dc in cat_sim.items():
    dset_dict = xs.extract_dataset(
        catalog=dc,
        region=region,
        xr_open_kwargs={"drop_variables": ["height", "time_bnds"]},
    )

    for key, ds in dset_dict.items():
        ds = xs.regrid_dataset(
            ds=ds,
            ds_grid=ds_grid,
            weights_location=str(output_folder / "gs-weights"),
            to_level="extracted",
        )
        filename = str(
            output_folder
            / f"wl_{ds.attrs['cat:id']}.{ds.attrs['cat:domain']}.{ds.attrs['cat:processing_level']}.{ds.attrs['cat:frequency']}.zarr"
        )
        chunks = xs.io.estimate_chunks(ds, dims=["time"], target_mb=50)
        xs.save_to_zarr(ds, filename, rechunk=chunks, mode="o")
        pcat.update_from_ds(ds=ds, path=filename, info_dict={"format": "zarr"})

## Find warming levels with only the model name

If all you want to know is the year or the period during which a climate model reaches a given warming level, then ``xs.get_warming_level`` is the function to use since you can simply give it a string or a list of strings and receive that information.

The arguments of ``xs.get_warming_level`` are:

- `realization`: Dataset, string, or list of strings. Strings should follow the format 'mip-era_source_experiment_member'
- `wl`: warming level.
- `window`: Number of years in the centered window during which the warming level is reached. Note that in the case of an even number, the IPCC standard is used (-n/2+1, +n/2).
- `tas_baseline_period`: The period over which the warming level is calculated, equivalent to "+0°C". Defaults to 1850-1900.
- `ignore_member`: The default `warming_level_csv` only contains data for 1 member. If you want a result regardless of the realization number, set this to True. This is only used when `models` is a Dataset.
- `return_horizon`: Whether to return the start/end of the horizon or to return the middle year.
    
If `realization` is a list, the function returns a dictionary. Otherwise, it will return either a string or ['start_yr', 'end_yr'], depending on `return_horizon`. For entries that it fails to find in the csv, or for instances where a given warming level is not reached, the function returns None.

In [ ]:
# Multiple entries, returns a dictionary
print(
    xs.get_warming_level(
        [
            "CMIP6_CanESM5_ssp126_r1i1p1f1",
            "CMIP6_CanESM5_ssp245_r1i1p1f1",
            "CMIP6_CanESM5_ssp370_r1i1p1f1",
            "CMIP6_CanESM5_ssp585_r1i1p1f1",
        ],
        wl=2,
        window=20,
        return_horizon=False,
    )
)
# Returns a list
print(
    xs.get_warming_level(
        "CMIP6_CanESM5_ssp585_r1i1p1f1", wl=2, window=20, return_horizon=True
    )
)
# Only the middle year is requested, returns a string
print(
    xs.get_warming_level(
        "CMIP6_CanESM5_ssp585_r1i1p1f1", wl=2, window=20, return_horizon=False
    )
)
# +10°C is never reached, returns None
print(xs.get_warming_level("CMIP6_CanESM5_ssp585_r1i1p1f1", wl=10, window=20))

## Find and extract data by warming levels

If you instead need to subset and analyze data, then two options are currently provided in `xscen`: `subset_warming_level` and `produce_horizon`. 
- Use `subset_warming_level` when you want to cut a dataset for a period corresponding to a given warming level, but its frequency untouched.
- Use `produce_horizon` when you need to: subset a time period, compute indicators, and compute the climatological mean for one or multiple horizons.

The two methods are detailed in the following section.

### Method #1: Subsetting datasets by warming level

``xs.subset_warming_level`` can be used to subset a dataset for a window over which a given global warming level is reached. Since the datasets will still have a 'time' axis after the subsetting, only a single warming level can be processed at a time, as to prevent overlapping `time` coordinates. A new dimension named `warminglevel` is created by the function, and thus it would in theory be possible to concatenate multiple results alongside that new dimension, but NaNs would need to be managed for subsequent steps.

The function calls `get_warming_level`, so the arguments are essentially the same.:

- `ds`: input dataset.
- `wl`: warming level.
- `window`: Number of years in the centered window during which the warming level is reached. Note that in the case of an even number, the IPCC standard is used (-n/2+1, +n/2).
- `tas_baseline_period`: The period over which the warming level is calculated, equivalent to "+0°C". Defaults to 1850-1900.
- `ignore_member`: The default `warming_level_csv` only contains data for 1 member. If you want a result regardless of the realization number, set this to True.
- `to_level`: Contrary to other methods, you can use "{wl}", "{period0}" and "{period1}" in the string to dynamically include `wl`, 'tas_baseline_period[0]' and 'tas_baseline_period[1]' in the `processing_level`.
- `wl_dim`: The string used to fill the new `warminglevel` dimension. You can use "{wl}", "{period0}" and "{period1}" in the string to dynamically include `wl`, `tas_baseline_period[0]` and `tas_baseline_period[1]`. If None, no new dimension will be added.
    
If the source, experiment, (member), and warming level are not found in the csv. The function returns None.

In [ ]:
ds = pcat.search(
    processing_level="extracted", experiment="ssp370", source="NorESM2-MM"
).to_dataset()

display(
    xs.subset_warming_level(
        ds,
        wl=1,
        window=20,
    )
)

### Method #2: Producing horizons

If what you need is to compute indicators and their climatological mean, ``xs.aggregate.produce_horizon`` is a more convenient function to work with than `subset_warming_level`. Since the years are meaningless for warming levels, and are even detrimental to making ensemble statistics, the function formats the output as to remove the 'time' and 'year' information from the dataset, while the seasons/months are unstacked to different coordinates. Hence, the single dataset outputed can contain indicators of different frequencies, as well as multiple warming levels or temporal horizons.

The arguments of ``xs.aggregate.produce_horizon`` are:

- `ds`: input dataset.
- `indicators`: As in `compute_indicators`
- `periods`: Periods to cut.
- `warminglevels`: Dictionary of arguments to pass to `subset_warming_level`. If 'wl' is a list, the function will be called for each value and produce multiple horizons.

If both periods and warminglevels are None, the full time series will be used. If a dataset does not contain a given period or warming level, then that specific period will be skipped.

In [ ]:
dict_input = pcat.search(processing_level="extracted").to_dataset_dict(
    xarray_open_kwargs={"decode_timedelta": False}
)

for id_input, ds_input in dict_input.items():
    # 2001-2020 will be used as our reference period. We can compute it at the same time.
    ds_hor = xs.produce_horizon(
        ds_input,
        indicators="samples/indicators.yml",
        periods=[["2001", "2020"]],
        warminglevels={"wl": [1, 1.5, 2], "window": 20, "ignore_member": True},
        to_level="horizons",
    )

    # Save
    filename = str(
        output_folder
        / f"wl_{ds_hor.attrs['cat:id']}.{ds_hor.attrs['cat:domain']}.{ds_hor.attrs['cat:processing_level']}.{ds_hor.attrs['cat:frequency']}.zarr"
    )
    xs.save_to_zarr(ds_hor, filename, mode="o")
    pcat.update_from_ds(ds=ds_hor, path=filename, info_dict={"format": "zarr"})

In [ ]:
display(ds_hor)

## Deltas and spatial aggregation

This step is done as in the [Getting Started](2_getting_started.ipynb#Computing-deltas) Notebook. Here we will spatially aggregate the data, but the datasets could also be regridded to a common grid.

In [ ]:
dict_wl = pcat.search(processing_level="horizons").to_dataset_dict(
    xarray_open_kwargs={"decode_timedelta": False}
)

for id_wl, ds_wl in dict_wl.items():
    # compute delta
    ds_delta = xs.aggregate.compute_deltas(
        ds=ds_wl, reference_horizon="2001-2020", to_level="deltas"
    )

    # remove the reference period from the dataset
    ds_delta = ds_delta.sel(horizon=~ds_delta.horizon.isin(["2001-2020"]))

    # aggregate
    ds_delta["lon"].attrs["axis"] = "X"
    ds_delta["lat"].attrs["axis"] = "Y"
    ds_agg = xs.spatial_mean(
        ds_delta,
        method="interp_centroid",
        to_level="deltas-agg",
        kwargs={"lon": -68, "lat": 49},
    )

    # Save
    filename = str(
        output_folder
        / f"wl_{ds_agg.attrs['cat:id']}.{ds_agg.attrs['cat:domain']}.{ds_agg.attrs['cat:processing_level']}.{ds_agg.attrs['cat:frequency']}.zarr"
    )
    xs.save_to_zarr(ds_agg, filename, mode="o")
    pcat.update_from_ds(ds=ds_agg, path=filename, info_dict={"format": "zarr"})

In [ ]:
display(ds_agg)

## Ensemble statistics

Even more than with time-based horizons, the first step of ensemble statistics should be to generate the weights. Indeed, if a model has 3 experiments reaching a given warming level, we want it to have the same weight as a model with only 2 experiments reaching that warming level. The argument `skipna=False` should be passed to `xs.generate_weights` to properly assess which simulations reaches which warming level. If the `horizon` dimension differs between datasets (as is the case here), they are reindexed and given a weight of 0.

When working with warming levels, how to assess experiments is more open-ended. The IPCC Atlas splits the statistics and climate change signals by SSPs, even when they are being analyzed through warming levels, but experiments could also be considered as 'members' of a given model and used to bolster the number of realizations. Note that `split_experiments=False` is the default in `xscen`.

In [ ]:
datasets = pcat.search(processing_level="deltas-agg").to_dataset_dict()

In [ ]:
# All realisations of NorESM2-MM are given the same weight for the first horizon, while it correctly recognizes that only the SSP585 reaches the 2nd horizon.
weights = xs.ensembles.generate_weights(
    datasets=datasets, independence_level="model", split_experiments=False, skipna=False
)
display(weights)

Next, the weights and the datasets can be passed to `xs.ensemble_stats` to calculate the ensemble statistics.

In [ ]:
ds_ens = xs.ensemble_stats(
    datasets=datasets,
    common_attrs_only=True,
    weights=weights,
    statistics={"ensemble_mean_std_max_min": None},
    to_level=f"ensemble-deltas-wl",
)

# It is sometimes useful to keep track of how many realisations made the ensemble.
ds_ens.horizon.attrs["ensemble_size"] = len(datasets)

filename = str(
    output_folder
    / f"wl_{ds_ens.attrs['cat:id']}.{ds_ens.attrs['cat:domain']}.{ds_ens.attrs['cat:processing_level']}.{ds_ens.attrs['cat:frequency']}.zarr"
)
xs.save_to_zarr(ds_ens, filename, mode="o")
pcat.update_from_ds(ds=ds_ens, path=filename, info_dict={"format": "zarr"})

In [ ]:
display(ds_ens)